In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import random
import string

In [3]:
%load_ext memory_profiler
%load_ext line_profiler

## 1. Read/write to a dict.
### 1.1 Read - existing key

In [4]:
# create and populate dict
d = {}
for i in range(1000):
    rand_num = random.randint(-255, 255)
    d[i] = rand_num

In [5]:
# set a function
def f(d, k):
    if k in d:
        return d[k]
    return None

In [6]:
%timeit d[456] #(1)
%timeit d.get(456, None) #(2)
%timeit d[456] if 456 in d else None #(3)
%timeit f(d, 456) #(4)
l = lambda d, k: None if k not in d else d[k]
%timeit l(d, 456) #(5)

70.1 ns ± 1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
114 ns ± 3.68 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
149 ns ± 6.3 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
216 ns ± 6.54 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
215 ns ± 10.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
%%timeit
try: # (6)
    d[456]
except KeyError:
    None

90 ns ± 4.64 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


доступ к существующему ключу:
1. самый быстрый способ (1) - просто получить доступ по ключу (хешу), при этом метод (6) так же быстр если ключ существует
1. в полтора раза медленнее (!) использование get (2) либо простой if (3) проверяющий тестируемый ключ на существование.
Полагаю что под капотом у get реализован примерно такой if
1. Вызовы именованной (4) и лямбда (5) функций работают практически одинаково плохо. Оверхед на их вызовы достаточно велик.

### 1.2 Read - Non existing key

In [8]:
%timeit d[2000] if 2000 in d else None #(1)
%timeit d.get(2000, None) #(2)
%timeit f(d, 2000) #(3)
l = lambda d, k: None if k not in d else d[k]
%timeit l(d, 2000) #(4)

61.9 ns ± 1.9 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
93.6 ns ± 1.45 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
158 ns ± 7.6 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
157 ns ± 4.06 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [9]:
%%timeit
#(5)
try:
    d[2000]
except KeyError:
    None

311 ns ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Проверка несуществующего ключа:
1. if для несуществующего ключа (1) работает так же быстро как обычный доступ по клюу при его наличии. Стоит заметить, что результат - вдвое быстрее, чем когда ключ существует. Вполне возможно это происходит из-за коллизий (два примера ниже, возможно, могут это подтвердить)
2. get (2) работает примерно за то же время что и при существующем ключе (немного быстрее). Это ожидаемо похоже на if
3. вызовы функций (3) и (4) все также работают меденнее чем предыдущие реализации. но при этом быстрее чем в случае с существующим ключом.
4. лидер антирейтинга (5) try except работает крайне медленно, вероятно из-за необходимости провести нетривиаьные действия по обработке исключения

In [10]:
# see, access to existing key is way slower. 
# this has to deal with (I assume) collisions and comparing keys themselves (when hasshes are the same).
# see below
%timeit 456 in d
%timeit 2000 in d

73.8 ns ± 2.52 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
58.4 ns ± 1.34 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [11]:
# this shows that with very little chance of collision access to existing vs non-existing key takes about the same time
a = {'veryuniquestring': 0, 'iamnotlikeanyoneinthisuniverse': 1}
%timeit 'veryuniquestring' in a
%timeit 'somerandomthing' in a

47.5 ns ± 1.46 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
49.2 ns ± 2.33 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


### 1.3 Write

In [12]:
%timeit d[1000] = 0 #(1)
%timeit d.setdefault(1001, 0) #(2)
%timeit d.update({1002: 0}) #(3)

75.9 ns ± 1.51 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
108 ns ± 1.58 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
255 ns ± 7.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Запись в словарь:
1. запись по ключу (1) - самый быстрый метод. равен доступу по ключу.
2. setdefault (2), видимо, содержит в себе накладные расходы на свой вызов и проверку на существование ключа (время сопоставимо с get
3. update (3) по всей видимости создает объект словаря прежде чем сливать. Отсюда такое время выполнения

### 1.4 Checking memory usage for dicts

In [13]:
%%writefile dict_set.py

from memory_profiler import profile

@profile
def track_dict_set():
    dict_plain = {}
    for i in range(1000, 10000):
        dict_plain[i] = i**10

    dict_set = {}
    for i in range(1000, 10000):
        dict_set.setdefault(i, i**10)

    dict_upd = {}
    for i in range(1000, 10000):
        dict_upd.update({i: i**10})

if __name__ == "__main__":
    track_dict_set()

Overwriting dict_set.py


In [14]:
!python dict_set.py

Filename: dict_set.py

Line #    Mem usage    Increment   Line Contents
     4     39.3 MiB     39.3 MiB   @profile
     5                             def track_dict_set():
     6     39.3 MiB      0.0 MiB       dict_plain = {}
     7     40.6 MiB      0.0 MiB       for i in range(1000, 10000):
     8     40.6 MiB      0.3 MiB           dict_plain[i] = i**10
     9                             
    10     40.6 MiB      0.0 MiB       dict_set = {}
    11     41.6 MiB      0.0 MiB       for i in range(1000, 10000):
    12     41.6 MiB      0.3 MiB           dict_set.setdefault(i, i**10)
    13                             
    14     41.6 MiB      0.0 MiB       dict_upd = {}
    15     42.6 MiB      0.0 MiB       for i in range(1000, 10000):
    16     42.6 MiB      0.3 MiB           dict_upd.update({i: i**10})




Судя по отчету профилировщика выше все три метода работают с (приблизительно) одинаковым потреблением памяти

## 2. List elements apply.
### 2.1 time exploring

In [15]:
experimental_list = [''.join(random.choices(string.ascii_lowercase, k=20)) for i in range(1000)]

In [16]:
def upper_list(l):
    res = []
    for i in l:
        res.append(i.upper())
    return res

def compare_list_updates(l):
    a = list(map(str.upper, l))
    b = [x.upper() for x in l]
    c = list(x.upper() for x in l)
    d = upper_list(l)

In [17]:
%lprun -f compare_list_updates -T str_list_upper_lprun.txt compare_list_updates(experimental_list)


*** Profile printout saved to text file 'str_list_upper_lprun.txt'. 


In [18]:
with open('str_list_upper_lprun.txt', 'r') as f:
    print(f.read())

Timer unit: 1e-07 s

Total time: 0.0034077 s
File: <ipython-input-16-9bb8126a32c0>
Function: compare_list_updates at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def compare_list_updates(l):
     8         1       2742.0   2742.0      8.0      a = list(map(str.upper, l))
     9         1       6962.0   6962.0     20.4      b = [x.upper() for x in l]
    10         1       9888.0   9888.0     29.0      c = list(x.upper() for x in l)
    11         1      14485.0  14485.0     42.5      d = upper_list(l)


используя для сравнительного анализа информацию из '% Time' можно увидеть что list-map самый быстрый способ (тоже скомпилирован на С?).
интересно что list comprehension быстрее вызова конструктора list с передачей ему генераторного выражения.
Аутсайдер - вызов самописной именованной функции которая итеративно наполняет вновь созданный результирующий лист.

In [19]:
l = experimental_list
%timeit a = list(map(str.upper, l))
%timeit b = [x.upper() for x in l]
%timeit c = list(x.upper() for x in l)
%timeit d = upper_list(l)

124 µs ± 28.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
112 µs ± 3.71 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
154 µs ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
152 µs ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


при использовании %timeit езультаты несколько отличаются. стоит отметить что результаты timeit в общем случае быстрее чем lprun (list-map занимает 108 vs 143.1 микросекунды)
1. разница между list-map и list comprehension незаметна.
2. list конструктор-генераторное выражение работает примерно так же как самописная именованная функция

### 2.2 Memory expolring

In [20]:
%memit?

In [21]:
%memit -r 3 list(map(str.upper, l))
%memit -r 3 [x.upper() for x in l]
%memit -r 3 list(x.upper() for x in l)
%memit -r 3 upper_list(l)

peak memory: 52.48 MiB, increment: 0.10 MiB
peak memory: 52.48 MiB, increment: 0.00 MiB
peak memory: 52.48 MiB, increment: 0.00 MiB
peak memory: 52.48 MiB, increment: 0.00 MiB


In [22]:
%memit g = (x.upper() for x in l)

peak memory: 52.48 MiB, increment: 0.00 MiB


результаты данного теста можно поставить под сомнение (перепроверить), тк все варианты дают абсолютно одинаковый результат. В моем представлении генераторное выражение должно возвращать сам генератор (являющийся ленивым). Те не брать сразу всю память (как описано по ссылке)
https://realpython.com/list-comprehension-python/#choose-generators-for-large-datasets

#### profiling numeric lists

In [34]:
l = [random.randint(-255, 255) for _ in range(100000)]

In [39]:
%%writefile num_list_funcs.py

import random
from memory_profiler import profile

@profile
def filter_list(l):
    c = [x for x in l if x > 0]
    a = filter(lambda x: x > 0, l)
    b = list(filter(lambda x: x > 0, l))
    del c, b
    return a

if __name__ == "__main__":
    l = [random.randint(-255, 255) for _ in range(100000)]
    filter_list(l)

Overwriting num_list_funcs.py


In [36]:
from num_list_funcs import filter_list

In [37]:
%lprun -f filter_list filter_list(l)

Мы видим что функция filter работает крайне быстро именно потому что возвращает итератор ничего по сути не выполняя.
Также видно, что использование list comprehension предпочтительнее чем filter в связке с конструктором list - время выполнения кратно меньше (х2)

при выполнении 'mprof run num_list_funcs.py' либо просто 'python num_list_funcs.py' из терминала видно что используемая память увеличивается на 0,4Мб 
на строке 10 (b = list(filter(lambda x: x > 0, l)))

## Sort list


In [41]:
l = [random.randint(-255, 255) for _ in range(100000)]

In [43]:
%timeit sorted(l)
%timeit l.sort()

1.41 ms ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
820 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


sorted конструирует новый список и поэтому разница в работу существенная. В остальном реализации должны быть идентичны

In [45]:
l = list(range(1000, -1, -1))
%timeit sorted(l)
l = list(range(0, 1001))
%timeit sorted(l)
l = list(random.randint(-255, 255) for _ in range(1000))
%timeit sorted(l)

10.2 µs ± 815 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
9.2 µs ± 389 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
102 µs ± 860 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


как видно сортировка уже отсортированного либо убывающего списка производится много быстрее случайной последовательности. для убывающей последовательности по видимому основная оптимизация в этом случае - descending run: https://en.wikipedia.org/wiki/Timsort#Descending_runs

In [75]:
%%writefile test_list_sort.py

import random
from memory_profiler import profile

@profile
def main():
    l = [random.randint(-255, 255) for _ in range(100000)]
    a = sorted(l)
    b = l.sort()
    
if __name__ == "__main__":
    main()

Overwriting test_list_sort.py


In [76]:
!python test_list_sort.py

Filename: test_list_sort.py

Line #    Mem usage    Increment   Line Contents
     5     39.5 MiB     39.5 MiB   @profile
     6                             def main():
     7     42.6 MiB      0.4 MiB       l = [random.randint(-255, 255) for _ in range(100000)]
     8     43.2 MiB      0.6 MiB       a = sorted(l)
     9     43.2 MiB      0.0 MiB       b = l.sort()




как и предполагалось, sorted вернул отсортированный список и занял дополнительные ~0,5 Мб. метод sort сделал все inplace

### Flatten list

In [33]:
l = [[1,2,3], 4, [5, 6, [7, 8, 9, [10, 11], 12, [13]]]] * 999

In [16]:
def get_for_loop_iterator(values):
    for i in values:
        if isinstance(i, list):
            yield from get_for_loop_iterator(i)
        else:
            yield i

In [111]:
%timeit list(get_for_loop_iterator(l))

585 µs ± 34.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
l = [[1,2,3], 4, [5, 6, [7, 8, 9, [10, 11], 12, [13]]]] * 999

In [35]:
def flatten_without_rec(non_flat):    
    flat = []    
    while non_flat: #runs until the given list is empty.        
        e = non_flat.pop()            
        if type(e) == list: #checks the type of the poped item.                
            non_flat.extend(e) #if list extend the item to given list.
        else:        
            flat.append(e) #if not list then add it to the flat list.            
    return flat

In [9]:
%timeit flatten_without_rec(l)

174 ns ± 8.43 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


функция на генераторах get_even_for_loop_iterator (из прошлого дз) работает почти в 3 раза медленнее чем решение с geeksforgeeks (flatten_without_rec). Причина, вероятно, в том, что flatten_without_rec не использует рекурсию и таким образом не тратит время на рекурсивные вызовы.

In [31]:
%%writefile test_flatten_list.py

from memory_profiler import profile


def get_for_loop_iterator(values):
    for i in values:
        if isinstance(i, list):
            yield from get_for_loop_iterator(i)
        else:
            yield i


def flatten_without_rec(non_flat):    
    flat = []    
    while non_flat: #runs until the given list is empty.        
        e = non_flat.pop()            
        if type(e) == list: #checks the type of the poped item.                
            non_flat.extend(e) #if list extend the item to given list.
        else:        
            flat.append(e) #if not list then add it to the flat list.            
    return flat

@profile
def main():
    l = [[1,2,3], 4, [5, 6, [7, 8, 9, [10, 11], 12, [13]]]] * 999    
    res1 = list(get_for_loop_iterator(l))
    l = [[1,2,3], 4, [5, 6, [7, 8, 9, [10, 11], 12, [13]]]] * 999
    res2 = flatten_without_rec(l)
    
if __name__ == "__main__":
    main()



Overwriting test_flatten_list.py


In [32]:
!python test_flatten_list.py

Filename: test_flatten_list.py

Line #    Mem usage    Increment   Line Contents
    23     39.4 MiB     39.4 MiB   @profile
    24                             def main():
    25     39.4 MiB      0.0 MiB       l = [[1,2,3], 4, [5, 6, [7, 8, 9, [10, 11], 12, [13]]]] * 999    
    26     39.7 MiB      0.3 MiB       res1 = list(get_for_loop_iterator(l))
    27     39.7 MiB      0.0 MiB       l = [[1,2,3], 4, [5, 6, [7, 8, 9, [10, 11], 12, [13]]]] * 999
    28     39.7 MiB      0.0 MiB       res2 = flatten_without_rec(l)




не совсем понятно почему, но профилировщик заявляет что память не выделяется ни на одном шаге больше чем в самом начале программы. Анна, если можете посмотреть где ошибка, буду очень признателен.
P.S. при увеличении с х99 до х999 размера входного массива видно что рекурсивная функция ест 0,3 Мб.